# Carvana U-Net Angles+Auxiliary

## Imports

In [1]:
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from optimizers.AdamAccumulate import AdamAccumulate
from models.u_net import UNet
from models.u_net_aux import UNet_Aux
from models.u_net_heng import UNet_Heng
from models.u_net_heng_aux import UNet_Heng_Aux
from utilities.submit import generate_submit
from utilities import utils_masks as utils
from utilities.losses import weighted_bce_dice_loss, dice_value

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## Preparing Data

In [1]:
utils.set_results_reproducible()
input_size = 128
num_pseudo_data = 3000
train_path = "inputs/train/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"
train_df = pd.read_csv('inputs/train_masks.csv')
test_path = "inputs/test/{}.jpg"
test_mask_path = "outputs/test_masks_2017-09-20-2223/{}.gif"
test_df = pd.read_csv('inputs/sample_submission.csv')

all_ids_train = train_df['img'].map(lambda s: s.split('.')[0])
all_ids_train_split, all_ids_valid_split = train_test_split(all_ids_train, test_size=0.2, random_state=42)

groups = ['01','02|16','03|15','04|14','05|13','06|12','07|11','08|10','09']
ids_train_splits = {}
ids_valid_splits = {}

for group in groups:
    train_df_group = train_df[(train_df.img.str.match('^.*_(' + group + ').jpg$'))]
    ids_group = train_df_group['img'].map(lambda s: s.split('.')[0])
    ids_train_split = pd.Series(list(set(all_ids_train_split).intersection(set(ids_group))))
    ids_valid_split = pd.Series(list(set(all_ids_valid_split).intersection(set(ids_group))))
    ids_train_splits[group] = ids_train_split
    ids_valid_splits[group] = ids_valid_split
    
    test_df_group = pd.DataFrame([])
    angles_group = group.split('|')
    for angle in angles_group:
        test_df_group = pd.concat([test_df_group, (test_df[(test_df.img.str.match('^.*_(' + angle + ').jpg$'))].sample(n=num_pseudo_data))])
    test_df_group = test_df_group.sample(n=num_pseudo_data * len(angles_group), replace=True)
    ids_test_split = test_df_group['img'].map(lambda s: s.split('.')[0])
    ids_test_splits[group] = ids_test_split
    
    print('group {0}:   #Training = {1}+{2}   #Validation = {3}'
          .format(group, len(ids_train_split), len(ids_test_split), len(ids_valid_split)))

bbox_file_path = 'inputs/data_bbox.csv'
bboxes = utils.get_bboxes(bbox_file_path)

def train_generator(batch_size, group, outputs=None):
    return utils.train_generator(train_path, train_mask_path, ids_train_splits[group], 
                                 input_size, batch_size, bboxes, outputs=outputs,
                                 augmentations=['HUE_SATURATION', 'SHIFT_SCALE', 'FLIP'])

def valid_generator(batch_size, group, outputs=None):
    return utils.valid_generator(train_path, train_mask_path, ids_valid_splits[group],
                                 input_size, batch_size, bboxes, outputs=outputs)

def pseudo_generator(batch_size, group, accum_iters, outputs=None):
    ids = utils.make_list_ids(ids_train_splits[group], ids_test_splits[group], batch_size, accum_iters)
    return utils.pseudo_generator(train_path, train_mask_path, test_path, test_mask_path, ids,
                                  input_size, batch_size, bboxes, outputs=outputs)

NameError: name 'utils' is not defined

## Create Model

In [3]:
def copy_weights(source, target):
    target_layers = [layer for layer in target.layers if not layer.name.startswith('aux_out')]
    for s_layer,t_layer in zip(source.layers, target_layers):
        #print('{0} --> {1}'.format(s_layer.name, t_layer.name))
        t_layer.set_weights(s_layer.get_weights())

In [4]:
#U-Net-Aux:
#model = UNet_Aux((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
#                 activation=lambda x: PReLU()(x), init='he_uniform', auxiliaries=[False, True, True, False])
#outputs = {'aux_out1':2**-1, 'aux_out2':2**-2, 'main_out':1}
#weights = {'aux_out1':0.2, 'aux_out2':0.05, 'main_out':1.}
#model.compile(optimizer=AdamAccumulate(accum_iters=4), 
#              loss=weighted_bce_dice_loss, metrics=[dice_value], loss_weights=weights)

#U-Net:
model = UNet((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
             activation=lambda x: PReLU()(x), init='he_uniform')
model.load_weights('weights/unet-2017-09-22-2306.hdf5')
model_aux = UNet_Aux((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
             activation=lambda x: PReLU()(x), init='he_uniform', auxiliaries=[False, True, True, False])
copy_weights(model, model_aux)
model_aux.save_weights('weights/unet-aux-base.hdf5')

#U-Net-Heng:
#model = UNet_Heng((input_size, input_size, 3))
#model.load_weights('weights/unet-heng-crop-2017-09-18-1053.hdf5')
#model_aux = UNet_Heng_Aux((input_size, input_size, 3),
#                          auxiliaries=[False, False, False, False, True, True])
#copy_weights(model, model_aux)
#model_aux.save_weights('weights/unet-heng-aux-base.hdf5')

## Fit Models

In [4]:
epochs = 150
batch_size = 1
auxiliaries = [False, True, True, False]
outputs = {'aux_out1':2**-1, 'aux_out2':2**-2, 'main_out':1}
weights = {'aux_out1':0.2, 'aux_out2':0.05, 'main_out':1.}

for group in groups:
    model = UNet_Aux((input_size, input_size, 3), filters=64, depth=4, dropout_base_only=False, dropout=0,
             activation=lambda x: PReLU()(x), init='he_uniform', auxiliaries=auxiliaries)
    model.compile(optimizer=AdamAccumulate(accum_iters=32),
                  loss=weighted_bce_dice_loss, metrics=[dice_value], loss_weights=weights)
    run_name = utils.get_run_name('weights/{}.hdf5', 'unet-aux-({})'.format(group))
    weights_path = 'weights/{}.hdf5'.format(run_name)

    callbacks = [EarlyStopping(monitor='val_main_out_dice_value',
                               patience=8,
                               verbose=1,
                               min_delta=1e-4,
                               mode='max'),
                 ReduceLROnPlateau(monitor='val_main_out_dice_value',
                                   factor=0.1,
                                   patience=4,
                                   verbose=1,
                                   epsilon=1e-4,
                                   mode='max'),
                 ModelCheckpoint(monitor='val_main_out_dice_value',
                                 filepath=weights_path,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='max'),
                 TensorBoard(log_dir='logs/{}'.format(run_name), batch_size=batch_size)]

    model.load_weights('weights/unet-aux-128-base.hdf5')
    K.set_value(model.optimizer.lr, 1e-4)

    print('Starting run "{}"'.format(run_name))
    model.fit_generator(generator=train_generator(batch_size, group, outputs),
                        steps_per_epoch=np.ceil(float(len(ids_train_splits[group])) / float(batch_size)),
                        epochs=epochs,
                        verbose=1,
                        callbacks=callbacks,
                        validation_data=valid_generator(batch_size, group, outputs),
                        validation_steps=np.ceil(float(len(ids_valid_splits[group])) / float(batch_size)))
    print()
    K.clear_session()
    sess = tf.Session()
    K.set_session(sess)

Starting run "unet-aux-(01)-2017-09-24-2258"
Epoch 1/150
259/259 [==============================] - 30s - loss: 0.5671 - aux_out1_loss: 0.5335 - aux_out2_loss: 0.5974 - main_out_loss: 0.4306 - aux_out1_dice_value: 0.9477 - aux_out2_dice_value: 0.9052 - main_out_dice_value: 0.9925 - val_loss: 0.5294 - val_aux_out1_loss: 0.4486 - val_aux_out2_loss: 0.5040 - val_main_out_loss: 0.4145 - val_aux_out1_dice_value: 0.9815 - val_aux_out2_dice_value: 0.9534 - val_main_out_dice_value: 0.9956
Epoch 2/150
259/259 [==============================] - 26s - loss: 0.5445 - aux_out1_loss: 0.4476 - aux_out2_loss: 0.4926 - main_out_loss: 0.4303 - aux_out1_dice_value: 0.9836 - aux_out2_dice_value: 0.9605 - main_out_dice_value: 0.9932 - val_loss: 0.5262 - val_aux_out1_loss: 0.4384 - val_aux_out2_loss: 0.4924 - val_main_out_loss: 0.4139 - val_aux_out1_dice_value: 0.9855 - val_aux_out2_dice_value: 0.9588 - val_main_out_dice_value: 0.9958
Epoch 3/150
259/259 [==============================] - 25s - loss: 0.5417

520/520 [==============================] - 50s - loss: 0.6117 - aux_out1_loss: 0.4977 - aux_out2_loss: 0.5113 - main_out_loss: 0.4866 - aux_out1_dice_value: 0.9867 - aux_out2_dice_value: 0.9770 - main_out_dice_value: 0.9941 - val_loss: 0.6027 - val_aux_out1_loss: 0.4955 - val_aux_out2_loss: 0.5208 - val_main_out_loss: 0.4776 - val_aux_out1_dice_value: 0.9869 - val_aux_out2_dice_value: 0.9709 - val_main_out_dice_value: 0.9959
Epoch 5/150
520/520 [==============================] - 49s - loss: 0.6101 - aux_out1_loss: 0.4962 - aux_out2_loss: 0.5068 - main_out_loss: 0.4855 - aux_out1_dice_value: 0.9869 - aux_out2_dice_value: 0.9790 - main_out_dice_value: 0.9941 - val_loss: 0.6024 - val_aux_out1_loss: 0.4949 - val_aux_out2_loss: 0.5140 - val_main_out_loss: 0.4778 - val_aux_out1_dice_value: 0.9872 - val_aux_out2_dice_value: 0.9751 - val_main_out_dice_value: 0.9958
Epoch 6/150
519/520 [============================>.] - ETA: 0s - loss: 0.6122 - aux_out1_loss: 0.4975 - aux_out2_loss: 0.5068 - ma

508/508 [==============================] - 48s - loss: 0.6037 - aux_out1_loss: 0.4904 - aux_out2_loss: 0.5014 - main_out_loss: 0.4805 - aux_out1_dice_value: 0.9851 - aux_out2_dice_value: 0.9761 - main_out_dice_value: 0.9931 - val_loss: 0.5937 - val_aux_out1_loss: 0.4869 - val_aux_out2_loss: 0.5028 - val_main_out_loss: 0.4711 - val_aux_out1_dice_value: 0.9854 - val_aux_out2_dice_value: 0.9738 - val_main_out_dice_value: 0.9948
Epoch 5/150
508/508 [==============================] - 49s - loss: 0.6044 - aux_out1_loss: 0.4911 - aux_out2_loss: 0.4989 - main_out_loss: 0.4812 - aux_out1_dice_value: 0.9853 - aux_out2_dice_value: 0.9781 - main_out_dice_value: 0.9932 - val_loss: 0.5934 - val_aux_out1_loss: 0.4867 - val_aux_out2_loss: 0.5006 - val_main_out_loss: 0.4710 - val_aux_out1_dice_value: 0.9855 - val_aux_out2_dice_value: 0.9750 - val_main_out_dice_value: 0.9948
Epoch 6/150
508/508 [==============================] - 49s - loss: 0.6062 - aux_out1_loss: 0.4920 - aux_out2_loss: 0.4976 - main_o

507/507 [==============================] - 50s - loss: 0.6031 - aux_out1_loss: 0.4907 - aux_out2_loss: 0.4978 - main_out_loss: 0.4800 - aux_out1_dice_value: 0.9870 - aux_out2_dice_value: 0.9801 - main_out_dice_value: 0.9943 - val_loss: 0.5917 - val_aux_out1_loss: 0.4871 - val_aux_out2_loss: 0.4995 - val_main_out_loss: 0.4693 - val_aux_out1_dice_value: 0.9872 - val_aux_out2_dice_value: 0.9782 - val_main_out_dice_value: 0.9961
Epoch 7/150
507/507 [==============================] - 50s - loss: 0.6031 - aux_out1_loss: 0.4903 - aux_out2_loss: 0.4959 - main_out_loss: 0.4802 - aux_out1_dice_value: 0.9871 - aux_out2_dice_value: 0.9811 - main_out_dice_value: 0.9942 - val_loss: 0.5912 - val_aux_out1_loss: 0.4868 - val_aux_out2_loss: 0.4976 - val_main_out_loss: 0.4690 - val_aux_out1_dice_value: 0.9873 - val_aux_out2_dice_value: 0.9791 - val_main_out_dice_value: 0.9962
Epoch 8/150
507/507 [==============================] - 50s - loss: 0.6023 - aux_out1_loss: 0.4903 - aux_out2_loss: 0.4953 - main_o

247/247 [==============================] - 24s - loss: 0.5497 - aux_out1_loss: 0.4482 - aux_out2_loss: 0.4698 - main_out_loss: 0.4366 - aux_out1_dice_value: 0.9863 - aux_out2_dice_value: 0.9746 - main_out_dice_value: 0.9939 - val_loss: 0.5368 - val_aux_out1_loss: 0.4422 - val_aux_out2_loss: 0.4801 - val_main_out_loss: 0.4244 - val_aux_out1_dice_value: 0.9868 - val_aux_out2_dice_value: 0.9681 - val_main_out_dice_value: 0.9956
Epoch 8/150
247/247 [==============================] - 24s - loss: 0.5523 - aux_out1_loss: 0.4493 - aux_out2_loss: 0.4702 - main_out_loss: 0.4389 - aux_out1_dice_value: 0.9864 - aux_out2_dice_value: 0.9751 - main_out_dice_value: 0.9938 - val_loss: 0.5361 - val_aux_out1_loss: 0.4410 - val_aux_out2_loss: 0.4757 - val_main_out_loss: 0.4241 - val_aux_out1_dice_value: 0.9873 - val_aux_out2_dice_value: 0.9701 - val_main_out_dice_value: 0.9957
Epoch 9/150
247/247 [==============================] - 24s - loss: 0.5477 - aux_out1_loss: 0.4461 - aux_out2_loss: 0.4654 - main_o

## Mask Prediction

In [ ]:
from PIL import Image
import cv2
from glob import glob

In [ ]:
model.load_weights('weights/unet-2017-09-02-1809.hdf5')

In [ ]:
imgs_names = glob('inputs/train/*.jpg')

In [ ]:
num = 1
input_size = 1024
pred_mask_path = "outputs/pred_train_masks/{}_mask.gif"
for i in range(len(imgs_names)):
    im = cv2.imread(imgs_names[i])
    im = cv2.resize(im, (input_size, input_size), interpolation=cv2.INTER_LINEAR)
    im = np.expand_dims(im, axis=0)
    im = np.array(im, np.float32) / 255
    #print im.shape
    pred_mask = model.predict(im, batch_size=1, verbose=2)
    #print pred_mask.shape
    pred_mask = np.squeeze(pred_mask, axis=[0, 3])
    #print pred_mask.shape
    pred_mask = cv2.resize(pred_mask, (1918, 1280), interpolation=cv2.INTER_LINEAR)
    #print pred_mask.shape
    pred_mask = (pred_mask > 0.95)
    pred_mask = Image.fromarray((pred_mask * 255).astype(np.uint8), mode='L')
    im_name = imgs_names[i].split('/')[-1]
    im_id = im_name.split('.')[0]
    pred_mask.save(pred_mask_path.format(im_id))
    
    if num%500==0:
        print '{0}/{1}'.format(num, len(imgs_names))
    num += 1

In [ ]:
#val_imgs, _ = next(valid_generator(len(ids_valid_split)))
train_imgs, _ = next(train_generator(len(ids_train_split[4000:])))

In [ ]:
#print val_imgs.shape
print train_imgs.shape

In [ ]:
#val_pred_masks = model.predict(val_imgs, batch_size=16, verbose=1)
train_pred_masks = model.predict(train_imgs, batch_size=16, verbose=1)

In [ ]:
preds = np.squeeze(train_pred_masks, axis=3)
np.shape(preds)

In [ ]:
pred_mask_path = "outputs/valid_masks/{}_mask.gif"

In [ ]:
for i in range(train_imgs.shape[0]):
    mask = preds[i]
    mask = cv2.resize(mask, (1918, 1280), interpolation=cv2.INTER_LINEAR)
    mask = (mask > 0.95)
    mask = Image.fromarray((mask * 255).astype(np.uint8), mode='L')
    mask.save(pred_mask_path.format(ids_valid_split.values[i]))

## Validation

In [ ]:
def np_dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

### Prediction

In [ ]:
run_name = 'unet-2017-09-03-1739'
model.load_weights('weights/{}.hdf5'.format(run_name))

val_imgs, val_masks = next(valid_generator(len(ids_valid_split)))
val_imgs = np.array(val_imgs)
val_masks = np.array(val_masks)
val_pred_masks = model.predict(val_imgs, batch_size=1)
masks_val_dices = [np_dice_value(mask, pred_mask) for (mask, pred_mask) in zip(val_masks, val_pred_masks)]

### Display the worst predicted mask for validation examples

In [ ]:
index = np.argsort(masks_val_dices)[7]
id = ids_valid_split.values[index]
utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(), show_img=False)
print id, masks_val_dices[index]

In [ ]:
indices = np.argsort(masks_val_dices[masks_val_dices <= 99.6])
for id in indices:
        print(masks_val_dices[id])

### Histogram

In [ ]:
hist, bins = np.histogram(masks_val_dices, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

### Visualization

In [ ]:
indices = np.random.randint(len(ids_valid_split), size=3)
for index in indices:
    id = ids_valid_split.values[index]
    utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(),
                    show_img=True, bbox = bboxes[id])

## Test

### Load Model

In [ ]:
# Create model first if required
run_name = 'unet-2017-08-20-5'
model.load_weights('weights/{}.hdf5'.format(run_name))

### Generate Submit

In [ ]:
batch_size = 16
threshold = 0.5
test_path = 'inputs/test1/' #'inputs/test/'
test_masks_path = 'outputs/test1_masks/' #None
generate_submit(model, input_size, batch_size, threshold, test_path, 'outputs/', run_name, test_masks_path)

### Visualization

In [ ]:
utils.show_test_masks(test_path, test_masks_path)